# A Webcam Based Laser Beam Analysis Tool

Webcams are flexible, inexpensive and often high resolution video cameras that can provide real-time image streaming to computers. By using image analysis tools on the image steam, a webcam can be converted into a tool for laser beam imaging and characterization. Since most beam analysis tools can be thousands of dollars, a webcam based alternative is an interesting prospect. 

We will implement three different beam analysis tools using a Logitech 720p webcam: real-time laser beam brightness, diameter, and centroid retrieval.

First, we import the necessary libraries for our webcam beam analyzer. We will use cv2 to capture video from our webcam, skimage to analyze the image stream, numpy for some helpful methods, and matplotlib and seaborn to visualize the data.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from skimage.morphology import opening, closing
from skimage.measure import moments
from skimage.segmentation import clear_border
from skimage import measure
from skimage.filters import threshold_otsu
from skimage.measure import regionprops, label

%matplotlib auto

sns.set_context("poster")
sns.set(style = "ticks")

Next, we instantiate the VideoCapture object from cv2, which will allow us to capture the camera's live stream. VideoCapture takes a device index as an argument, where 0 is the computer's default device index (e.g. a Mac's built in webcam) and 1 is the next device index, etc. To access the Logitech webcam plugged into a USB port, we use device index 1. We also disable the webcam's auto exposure and autofocus properties.

In [ ]:
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.0)
cap.set(cv2.CAP_PROP_AUTOFOCUS, 0.0)
#cap.set(cv2.CAP_PROP_GAIN, 0.0)

To check that everything is working, we want to look at our webcam feed. We use .read( ) to capture images, frame-by-frame, where response is a boolean value that returns True if the frame is read in correctly. We can capture images in a loop, convert them to grayscale using the .cvtColor( ) method and display the image feed using .imshow( ) and .pause( ) (which pauses for an interval before updating the active figure).

In [ ]:
response, frame = cap.read()
ax = plt.gca()
while response:
    ax.clear()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    plt.imshow(frame, cmap = 'gray')
    plt.pause(0.05)
    response, frame = cap.read()

Next we define two functions:

The first function, thresh_close_open, takes an image and a scalar, and thresholds an image (where the thresholding can be scaled by the scalar argument), clears the border, and then closes and opens the image before returning it. 

The second function, max_contour_diam, takes a binary image, retrieves its contours using the cv2 method .findContours( ), draws the filled contours using the cv2 method .drawContours( ), and then retrieves the largest diameter contour in the image. The average diameter of this largest contour is then determined by looking at the average distance between the contour and the image centroid, which is determined using the image moments. 

In [ ]:
def thresh_close_open(image, thresh_scale):
        
    thresh = threshold_otsu(image)
    binary = image > thresh * thresh_scale
    
    image_border = clear_border(binary)

    close_ = closing(image_border)
    open_ = opening(close_)
    
    return open_

def max_contour_diam(image):
    
    _, contours, heirarchy = cv2.findContours(image.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    image = cv2.drawContours(image.astype(np.uint8), contours, -1, (127,127,127), 5)
    
    sort_cont = sorted(contours, key = cv2.contourArea)
    largest = sort_cont[-1].tolist()
    largest = sum(largest, [])
    
    M = moments(image)
    x, y = M[1, 0] / M[0, 0], M[0, 1] / M[0, 0]
    
    norms = [np.sqrt((lar[0] - x)**2 + (lar[1] - y)**2) for lar in largest]
    norm = np.mean(norms)
    diameter = 2 * norm
    
    return diameter

Now we are ready to construct and deploy our first beam analysis tools: the average beam brightness and diameter retrieval tool. As above, we use the .read( ) function and a loop to capture images frame-by-frame, were we apply our image analysis to each frame. 

Here, we convert the color image to grayscale and compute the average brightness of the frame using np.mean( ). We then record this number by appending it to a list so the data can be plotted in real-time. Using an if statement, we ensure that the list does not exceed a length of 90, which allows us create a moving window to view the trend in the data over the past 90 frames. 

Next, we use our function thresh_close_open to return the frame after thresholding and opening and closing. We then feed this frame into our function max_contour_diam to back out the maximum contour diameter in the frame, which we then record by appending it to a list. This data is also visualized in real-time using an if statement to create another moving window to view the data trend over the past 90 frames, just as before. This data is then visualized using matplotlib and the .pause( ) function.

In [ ]:
response, frame = cap.read()

num = 90
spacer = 10
data = []
diam = []
i = 0

fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (6, 4.5))

while response:
    
    ax1.cla()
    ax2.cla()
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    ave = np.mean(frame)
    if len(data) > num:
        data.pop(0)
        i = 90
    data.append(ave)
    
    open_ = thresh_close_open(frame, 1.0)
    
    #_, contours, heirarchy = cv2.findContours(open_.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #open_ = cv2.drawContours(open_.astype(np.uint8), contours, -1, (127,127,127), 5)
    
    #diameter = max_contour_diam(contours, frame)
    diameter = max_contour_diam(open_)
    if len(diam) > num:
        diam.pop(0)
        i = 90
    diam.append(diameter)
    
    ax1.set_xlim(0, num + spacer)
    ax1.set_xticklabels([])
    ax1.set_title('Brightness: {:06.3f}'.format(ave))
    #ax1.set_title('Diameter: {:06.3f}'.format(diameter))
    ax1.tick_params(axis = 'x', bottom = False, labelbottom = False)
    ax1.plot(data)  #, label = 'Average Power: {:06.3f}'.format(ave))
    ax1.scatter(i, ave, alpha = 0.9, s = 100)
    #ax1.legend()

    
    ax2.set_xlim(0, num + spacer)
    ax2.set_xticklabels([])
    ax2.set_title('Diameter: {:06.3f}'.format(diameter))
    ax2.tick_params(axis = 'x', bottom = False, labelbottom = False)
    ax2.plot(diam)  #, label = 'Diameter: {:06.3f}'.format(diameter))
    ax2.scatter(i, diameter, alpha = 0.9, s = 100)
    #ax2.legend()

    fig.tight_layout()
    plt.pause(0.05)
    
    response, frame = cap.read()
    
    i += 1

Next we can construct and deploy our beam centroiding tool. As before, we read in images frame-by-frame using the .read( ) function, which we convert to grayscale. Once we have a grayscale image, we apply our function thresh_close_open to the frame, before passing the modified frame to skimage's label( ) function, which labels all simply connected regions in the frame. To centroid the beam, we then loop over the regions retrieved by label( ) and check if a connected region is large enought to be a beam. If the beam is detected, the beam centroid, area and eccentricity is computed and then displayed on the original frame, where the centroid is marked on the frame by a red '+' symbol, and the area and eccentricity are displayed as the title. For comparison, we display both the original frame with the computed properties displayed, as well as the thresholded and closed and opened frame, by using np.hstack( ) to horizontally stack the two frame arrays.

In [ ]:
response, frame = cap.read()

fig, ax = plt.subplots(1, 1, figsize = (6, 4.5))

while response:
    
    ax.cla()
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #thresh = threshold_otsu(frame)
    #binary = frame > thresh * 1.7

    #frame_border = clear_border(binary)
    
    #close_ = closing(frame_border)
    #open_ = opening(close_)
    
    open_ = thresh_close_open(frame, 1.7)
    
    frame_regions = label(open_)
    
    #loop over the properties of labeled regions
    for region in regionprops(frame_regions):
    
        #check if the region is big enough to be a beam
        if region.area >= 500:
                    
            cent = region.centroid    
            #cent = region.weighted_centroid
            area = region.area
            ecc = round(region.eccentricity, 3)
            ax.text(cent[1], cent[0], '+', fontsize = 14,
                horizontalalignment='center', verticalalignment='center', color = 'red',
                fontdict = {'family': 'sans-serif', 'weight': 'bold'})
            ax.set_title('Centroid: ({:02.1f},{:02.1f}), '.format(*cent) + 'Eccentricity: {:02.1f}, '.format(ecc) + 
                         'Area: {:02.1f}'.format(area), color = 'black',
                fontdict = {'family': 'sans-serif', 'weight': 'bold', 'fontsize': 14})
    
    open_ = open_.astype(np.uint8)
    open_[open_ == True] = 255
    open_[open_ == False] = 0
    stack_images = np.hstack([frame, open_])
    plt.tight_layout()
    plt.imshow(stack_images, cmap = 'gray')
    plt.pause(0.1)
    response, frame = cap.read()

In [ ]:
response, frame = cap.read()
ax = plt.gca()
while response:
    ax.clear()
    cap.set(cv2.CAP_PROP_GAIN, 0.0)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    thresh = threshold_otsu(frame)
    binary = frame > thresh

    close_ = closing(binary)
    open_ = opening(close_)
    
    _, contours, heirarchy = cv2.findContours(open_.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    open_ = cv2.drawContours(open_.astype(np.uint8), contours, -1, (127,127,127), 5)

    plt.imshow(open_, cmap = 'gray')
    plt.pause(0.05)
    response, frame = cap.read()